<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 36</h1>
</div>

# 1. Multithreading, concurrency, parallelism

# 2. Simple multithreading: crate `rayon`

# 3. Other things available in Rust and in general

# <font color="red">1. Multithreading, concurrency, parallelism</font>

# 2. Simple multithreading: crate `rayon`

# 3. Other things available in Rust and in general

## Running multiple things at once

Various reasons:

* Separate GUI from the processing engine in your application
  * more responsive user experience

* Big scale computation: solving big data problems

* Running analytics on your laptop:
  * speeding up a single core more and more challenging
  * more cores even in consumer laptops
  

* GPUs offer **a lot** of (restricted) parallelism

## Term explanation

* Parallelism: things running at the very same time, different cores, processors, machines 

* Concurrency: the art of sharing resources, even if only one thread is running at a time

* Threads:
 * minimum organizational unit of your computation on a single machine
 * multiple of them allowed, running at the same or different times

## Solving a given problem more efficiently via parallel computation?

* Very problem dependent:

<div align="center">
    <b>digging the Suez canal&nbsp;&nbsp;&nbsp;&nbsp;vs.&nbsp;&nbsp;&nbsp;&nbsp;digging a deep well</b>
</div>

* What is possible: one of the deepest questions in computer science

## Programming: difficult and very error–prone

Challenges: 

* Information exchange

* Sharing resources

* Taking and returning them properly:
  * Similar to challenges in memory management

## Dining philosophers' problem

* Multiple philosophers sitting around the table
  * they do two things: think and eat

* A single fork between each two of them

* A philosopher needs two forks to eat

<div align="center">
    <img src="philosophers.png" alt="[philosophers]" width="50%">
</div>

<div align="center">
    <b>What algorithm could the philosophers use to achieve their life goals: eating and thinking?</b>
</div>

## Potential problems

How about this algorithm?

```
repeat:
   think
   take left fork when available
   take right fork when available
   eat
   return left fork
   return right fork
```

* All philosophers could reach for the left fork at the same time!

* They are all stuck

* This is called **deadlock**

## Potential problems

How about this algorithm?

```
repeat:
   think
   take any of the forks
   if the other available:
       take it
       eat
   return all forks you have 
```

* A philosopher may never eat!

* This is called **starvation**

# 1. Multithreading, concurrency, parallelism

# <font color="red">2. Simple multithreading: crate `rayon`</font>

# 3. Other things available in Rust and in general

## Crate `rayon`

General case difficult:
  * manual management of threads
  * communication and sharing work by them

Often you may want to speed up simple tasks:
  * sorting
  * a loop with independent iterations 
  
(Similar in many ways to OpenMP for C/C++/Fortran)

## Auxiliary definitions

In [ ]:
:dep rayon
:dep rand
use rayon::prelude::*;
use std::thread;
use std::time::{Duration,SystemTime};
use rand::Rng;
use std::time::

// see how long something is executing
fn execute_it(f:fn(u64)->(), id:usize, millis:u64) {
    let before = SystemTime::now();
    f(millis);
    let after = SystemTime::now();
    println!("Thread {} Time: {:.3?}", id, after.duration_since(before).unwrap())
}

// do nothing for a specific number of milliseconds
fn wait(millis:u64) {
    std::thread::sleep(Duration::from_millis(millis));
}

let mut handles = vec![];
for i in 0..10 {
    let handle = thread::spawn(move || {
        execute_it(wait, i, 1000);
    });
    handles.push(handle);
}
for handle in handles {
    handle.join().unwrap();
}

## Example of sorting

In [ ]:
fn time_it(f: impl FnOnce() -> ()) {
    let before = SystemTime::now();
    f();
    let after = SystemTime::now();
    println!("Time: {:.3?}", after.duration_since(before).unwrap())
}

// random 
const N: usize = 30_000_000;
let mut v = Vec::new();
for i in 0..N {
    v.push(rand::thread_rng().gen_range(0..(N as i32)));
};

In [18]:
let mut v_copy = v.clone();
time_it(|| v_copy.sort_unstable());

let mut v_copy = v.clone();
time_it(|| v_copy.sort());

Time: 847.172ms
Time: 2.063s


In [19]:
let mut v_copy = v.clone();
time_it(|| v_copy.par_sort_unstable());

let mut v_copy = v.clone();
time_it(|| v_copy.par_sort());

Time: 139.276ms
Time: 301.458ms


## Replacing iterators with parallel iterators

Replace `iter()` with `par_iter()`, `into_iter()` with `into_par_iter()`, etc.

In [20]:
// standard version
(1..=20).for_each(|x| {println!("{}",x);});

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


## Replacing iterators with parallel iterators

Replace `iter()` with `par_iter()`, `into_iter()` with `into_par_iter()`, etc.

In [21]:
// add explicit iterator construction
(1..=20).into_iter().for_each(|x| {println!("{}",x);});

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


## Replacing iterators with parallel iterators

Replace `iter()` with `par_iter()`, `into_iter()` with `into_par_iter()`, etc.

In [22]:
// replace into_iter() with into_par_iter() and wait for 500 ms to slow things down
(1..=20).into_par_iter().for_each(|x| {wait(500); println!("{}",x);});

1
11
3
6
2
8
7
9
16
4
12
10
18
19
17
13
20
14
15
5


## Replacing iterators with parallel iterators

Replace `iter()` with `par_iter()`, `into_iter()` with `into_par_iter()`, etc.

In [23]:
// make the wait time variable to see other patterns of execution
(1..=20).into_par_iter().for_each(|x| {wait(x*x*10); println!("{}",x);});

1
2
3
4
6
5
8
9
7
10
11
12
13
14
16
15
18
17
19
20


## Benchmarking parallel processing of a long vector

In [24]:
let mut v1 : Vec<i32> = (1..=50_000_000).collect();
let mut v2 = v1.clone();

In [25]:
// non-parallel version
time_it(|| v1.iter_mut().for_each(|x| *x += 100 / *x + *x / 100));

Time: 41.776ms


In [26]:
// using parallel iterators
time_it(|| v2.par_iter_mut().for_each(|x| *x += 100 / *x + *x / 100));

Time: 13.228ms


# 1. Multithreading, concurrency, parallelism

# 2. Simple multithreading: crate `rayon`

# <font color="red">3. Other things available in Rust and in general</font>

## Sample other things available in Rust and beyond

* Starting separate threads and channels to share data
  * communicating to share data:
    * data sent between threads via channels
    * data that was transmitted cannot be accessed anymore:
      * verification via Rust's ownership rules
      * checked at compile time!

* Mutex
  * a lock for accessing a specific resource
  * various versions:
    * only one thread has access at a time
    * or multiple threads with read access / only one thread with write access

* Chapter 16 of "The Rust Programming Language": overview of some mechanisms available in Rust
* More on parallel programming and Rust support for it next lecture